In [9]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES']="6"
project_root = os.path.abspath(os.path.join("/data/fengjw/Project/DAMA/DAMA"))
if project_root not in sys.path:
    sys.path.append(project_root)

In [10]:
import torch
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, process_images, get_model_name_from_path
from matplotlib import pyplot as plt

In [3]:
from datasets import load_dataset
from PIL import Image
import json
from tqdm import tqdm
import numpy as np
import io
import pandas as pd

In [11]:
# model_path = "/NAS/fengjw/.cache/output/llava7b_repo_model/checkpoint-2500"
model_path = "/NAS/fengjw/.cache/output/DAMA/main/llava7b_dpo_01"
model_base = None

model_path = os.path.expanduser(model_path)
model_name = get_model_name_from_path(model_path)

In [20]:
print('Loading Model...')
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)

Loading Model...


Loading checkpoint shards: 100%|██████████| 2/2 [03:52<00:00, 116.50s/it]
Some weights of the model checkpoint at /NAS/fengjw/.cache/output/DAMA/main/llava7b_dpo_01 were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.4.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.22.self_attn.k_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.19.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.2.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.14.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.6.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.9.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.3.layer_norm1.weight', 'model.vision

In [2]:
def bytes_to_PIL_image(img_buffer):
    img_io = io.BytesIO(img_buffer)
    img_io.seek(0)
    image = Image.open(img_io).convert('RGB')
    return image

In [38]:
# ds = load_dataset("/NAS/fengjw/.cache/base_datasets/test_522/RLAIF-V-Dataset_22k_logps")['train']
ds = load_dataset("/NAS/fengjw/.cache/base_datasets/test_522/RLAIF-V-Dataset_22k_logps_candidates_1")['train']
# ds_new = load_dataset("/NAS/fengjw/.cache/base_datasets/test_522/RLAIF-V-Dataset_22k_logps_candidates_1")['train']

Extracting data files: 100%|██████████| 1/1 [00:01<00:00,  1.83s/it]
Generating train split: 6000 examples [00:12, 485.51 examples/s]


In [37]:
ds[8000]

{'ds_name': 'RLAIF-V',
 'image': {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x02\x02\x03\x02\x02\x02\x02\x02\x04\x03\x03\x02\x03\x05\x04\x05\x05\x05\x04\x04\x04\x05\x06\x07\x06\x05\x05\x07\x06\x04\x04\x06\t\x06\x07\x08\x08\x08\x08\x08\x05\x06\t\n\t\x08\n\x07\x08\x08\x08\xff\xdb\x00C\x01\x01\x01\x01\x02\x02\x02\x04\x02\x02\x04\x08\x05\x04\x05\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\xff\xc0\x00\x11\x08\x02\x80\x01\xe0\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x02\x02\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x05\x06\x04\x07\x03\x08\x01\x02\t\x00\n\x0b\xff\xc4\x00J\x10\x00\x02\x01\x02\x05\x02\x05\x01\x05\x06\x04\x05\x03\x03\x00\x0b\x01\x02\x03\x04\x11\x00\x05\x06\x12!1A\x07\x13"Qaq\x08\x142\x81\x91\x15

In [8]:
ds_new[0]

{'ds_name': 'RLAIF-V',
 'image': {'bytes': b'RIFFH\x90\x00\x00WEBPVP8 <\x90\x00\x00P\x0c\x04\x9d\x01*\xb0\x04\x9b\x02>\x9dJ\x9fL%\xab\xbe2\xa5\x12:\x83\xc0\x13\x89enHjL\xfb\xff\xfb6\x97B\xdbD\xe7\xcbjK\xc1\xcf5\xe6\xec\xff\xd9\x9c\xb1\xf4\x07\x8a?\xe2xZ\xfa\xcf\xfb.\x9d\xcf\xf6\xbb\xd7\xbc\xc7\xa8w\x1f{St\x8c\xff\xc3\xcd\xe8khu\x9b\xb7S\xb8\x8a\xfd\xed\x9f\xd3\x89\t\xc4?\xf8\xe7N\x7f:\xfe\xaf:tc\xa0^\xb0\xfcO\xffw\xa0\xef\x92\x7f\x9d\xfb{\xe5\xffj\xe9\xfa\xd5\x7f\x95\xbei\xfdi\x9d\x7f&\xee\xcb;C\xf6\xf2\x00sB<N\xd5\xa7)\xc41\x95\xea\xa1\xcd\xe3\x98\x9f\xb5\xee\xbbk\x94\x13\xf0\x90a\xc1\xe8\xf4*\xde\x18\x9c\xd9G\xdcz\xbb\xb2\x1c>\xfa\xdb\xde\xe2\xc0\x8f!\x07\xc8|At\x82\xc0\xdfk\xb2W\x18\x0f4:\xbeiZ\xb3\x98~q\xb5s\x0b\x1aT\x80\x1epnU\xb2\x9a\x13\xa8\xafnM\xb6\x0cu\xdf\\4\xcc7S\xc58\xc4Y\x8aH\x9d\x91\\\xcc"=\xd7\xcd\x159\x8cQ\x11k\xe0\x9d\xdc[}\xf0Ezw\xee\x9b\xcd:\xea\x01\xc1}\xe0\xcfC*\xa0 RiX6\xa7\xc8\xd8\\HO\xb0\xd4*\x7f\xcb`\x1c.\xb4\xaaX9U\xbf\xbc\x1dD\xae"TbB\xab\x90g\xdd\x93\xe45\x

In [6]:
sample = ds[0]

In [7]:
sample

{'ds_name': 'RLAIF-V',
 'image': {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x02\x02\x02\x02\x02\x02\x02\x02\x02\x02\x02\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x02\x02\x01\x02\x02\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\xff\xc2\x00\x11\x08\x03 \x04\xb0\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x008\x00\x00\x01\x04\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x03\x04\x05\x07\x02\x08\t\x01\x00\n\x01\x00\x02\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x01\x02\x05\x00\x06\x07\x08\xff\xda\x00\x0c\x03\x01\x00\x02\x10\x03\x10\x00\x00\